# Análisis inicial y trata de datos

In [1]:
# set PYTHONPATH="${PYTHONPATH}:/path/to/your/project/"

In [2]:
# Importamos las librerías necesarias
import pandas as pd # type: ignore
import numpy as np # type: ignore
# from src import funciones_soporte as fs
import psycopg2 as ps # type: ignore

In [3]:
# Importamos el archivo con el que vamos a trabajar, que en este caso es un parquet. Para que funcione el read_parquet es necesario pasarle
# el parámetro ' engine = auto/pyarrow', ambos funcionan. En caso de que se use pyarrow será necesario realizar su instalaciñon previamnente en el entorno.
df_raw = pd.read_parquet("../data/reservas_hoteles.parquet", engine='auto')
df_raw.head()

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad
0,5256cc90-139b-43d2-8ec5-412495d751cf,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Feliciana,Cantón,feliciana.cantón@example.com,True,,2025-03-01,2025-03-02,113,NaN,,NaN,
1,84fd6209-bd8d-4a92-bfe6-c68ee2c49271,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Leonardo,Hierro,leonardo.hierro@example.com,True,,2025-03-01,2025-03-02,194,NaN,,NaN,
2,40c4cb55-d1f5-407b-832f-4756b8ff77b4,6ec4d7d2-e382-47cc-94f6-c647577d4035,Maite,Calatayud,maite.calatayud@example.com,False,2025-02-09,2025-03-01,2025-03-02,14,119.96,Hotel Monte Verde,1.0,Madrid
3,f2ce8df5-7844-43e1-8c0f-97ce7a208f21,aed17fe8-eb18-4f69-bf08-df1afdc339c3,Tecla,Bonet,tecla.bonet@example.com,False,2025-02-08,2025-03-01,2025-03-02,38,211.53,Hotel Brisas del Mar,5.0,Madrid
4,57d4515a-447d-4067-afcb-9bdcf4e4e915,710a2fbb-264f-426a-aa66-f8fffd146a2d,Amílcar,Andrés,amílcar.andrés@example.com,False,2025-02-02,2025-03-01,2025-03-02,35,378.37,Hotel Camino del Sol,1.0,Madrid


In [4]:
# Creamos una copia de los datos originales
data = df_raw.copy()
data.head()

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad
0,5256cc90-139b-43d2-8ec5-412495d751cf,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Feliciana,Cantón,feliciana.cantón@example.com,True,,2025-03-01,2025-03-02,113,NaN,,NaN,
1,84fd6209-bd8d-4a92-bfe6-c68ee2c49271,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Leonardo,Hierro,leonardo.hierro@example.com,True,,2025-03-01,2025-03-02,194,NaN,,NaN,
2,40c4cb55-d1f5-407b-832f-4756b8ff77b4,6ec4d7d2-e382-47cc-94f6-c647577d4035,Maite,Calatayud,maite.calatayud@example.com,False,2025-02-09,2025-03-01,2025-03-02,14,119.96,Hotel Monte Verde,1.0,Madrid
3,f2ce8df5-7844-43e1-8c0f-97ce7a208f21,aed17fe8-eb18-4f69-bf08-df1afdc339c3,Tecla,Bonet,tecla.bonet@example.com,False,2025-02-08,2025-03-01,2025-03-02,38,211.53,Hotel Brisas del Mar,5.0,Madrid
4,57d4515a-447d-4067-afcb-9bdcf4e4e915,710a2fbb-264f-426a-aa66-f8fffd146a2d,Amílcar,Andrés,amílcar.andrés@example.com,False,2025-02-02,2025-03-01,2025-03-02,35,378.37,Hotel Camino del Sol,1.0,Madrid


In [5]:
# función info
def info_df(dataframe):
    """
    Función que devuelve información general sobre el DatFrame que le pasemos.


    Args:
        df (DataFrame): DataFrame con información que queramos revisar

    Returns:
        DataFrame: DataFrame con información general sobre las columnas del DataFrame que se le ha pasado a la función: tipo de datos, número de
        registros, número de valores nulos, porcentaje de los valores nulos sobre el total
    """
    info_df = pd.DataFrame()
    info_df["Tipo_dato"] = dataframe.dtypes
    info_df["numero_registros"] = [dataframe[elemento].value_counts().sum() for elemento in dataframe]
    info_df["Numero_nulos"] = round(dataframe.isnull().sum())
    info_df["%_nulos"] = round((dataframe.isnull().sum()/dataframe.shape[0])*100, 2)

    return info_df

In [6]:
# Información general del dataframe
info_df(data)

,Tipo_dato,numero_registros,Numero_nulos,%_nulos
id_reserva,object,15098,0,0.00
id_cliente,object,15098,0,0.00
nombre,object,15098,0,0.00
apellido,object,15098,0,0.00
mail,object,15098,0,0.00
competencia,bool,15098,0,0.00
fecha_reserva,object,15098,0,0.00
inicio_estancia,object,15023,75,0.50
final_estancia,object,15023,75,0.50
id_hotel,int64,15098,0,0.00


In [7]:
# Vamos a ver las columnas que tenemos en el dataframe
data.columns

Index(['id_reserva', 'id_cliente', 'nombre', 'apellido', 'mail', 'competencia',
       'fecha_reserva', 'inicio_estancia', 'final_estancia', 'id_hotel',
       'precio_noche', 'nombre_hotel', 'estrellas', 'ciudad'],
      dtype='object')

Tras realizar una overview de los datos recibidos observamos las siguientes casuísticas:
- Tenemos 14 columnas.
- Hay 3 columnas con información sobre fechas, las cuales son de tipo object. Será necesario modificar el tipo de dato de las mismas para poder trabajar más adelante.
- La columna competencia es un booleano, el cual habrá que ver con que se corresponde true y false.
- Tenemos tres columnas de id: de reserva, de cliente y de hotel. Las dos primeras con tipo object, y la última con tipo int. Cambiaremos la de id_hotel a tipo object para tenerlas las tres iguales. 
- Hay existencia de nulos en cuatro columnas:
    
    **a)** inicio_estancia: tenemos un 0,5% de nulos.
    
    **b)** fin_estancia: tenemos un 0,5% de nulos. Comprobaremos si son los mismos que la columna de inicio estancia, revisar casuística y tratar.
    
    **c)** precio_noche: hay un 34,6% de nulos, un valor significativo. Será necesario tratarlos.
    
    **d)** estrellas: hay un 34,26% de nulos, un valor significativo. Será necesario tratarlos. Asimismo el tipo de dato de esta columna es float. Podemos plantear modificarlo a tipo object, a un str. 

A continuación realizamos las modificaciones necesarias de las columnas para poder trabajar con ellas. 

## Búsqueda de duplicados completamente iguales 

In [43]:
# pd.set_option('display.max_rows', None)

In [72]:
# Vamos a revisar la existencia de duplicados completamente iguales en el df. Revisaremos si se pueden eliminar
data.duplicated().sum()
# Tenemos 98 elementos duplicados en total en el dataframe
data = data.drop_duplicates()

## Modificación columnas de fecha

In [73]:
# Creamos una función que nos modifica ek tipo de dato de las columnas que le pasemos a tipo fecha
def data_fechas(dataframe, columnas):
    for col in columnas:
        dataframe[col] = pd.to_datetime(dataframe[col])
    return dataframe

In [74]:
# modificamos las columnas que queremos a tipo fecha
data = data_fechas(data, ["fecha_reserva", "inicio_estancia", "final_estancia"])

In [75]:
# comprobamos que la modificación se ha realizado de forma correcta
info_df(data)

,Tipo_dato,numero_registros,Numero_nulos,%_nulos
id_reserva,object,15000,0,0.00
id_cliente,object,15000,0,0.00
nombre,object,15000,0,0.00
apellido,object,15000,0,0.00
mail,object,15000,0,0.00
competencia,bool,15000,0,0.00
fecha_reserva,datetime64[ns],9828,5172,34.48
inicio_estancia,datetime64[ns],14925,75,0.50
final_estancia,datetime64[ns],14925,75,0.50
id_hotel,object,15000,0,0.00


Modificación realizada con éxito.

## Columna valores booleanos

In [76]:
data.head()

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad,col_prueba
0,5256cc90-139b-43d2-8ec5-412495d751cf,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Feliciana,Cantón,feliciana.cantón@example.com,True,NaT,2025-03-01,2025-03-02,113,NaN,,NaN,,True
1,84fd6209-bd8d-4a92-bfe6-c68ee2c49271,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Leonardo,Hierro,leonardo.hierro@example.com,True,NaT,2025-03-01,2025-03-02,194,NaN,,NaN,,True
2,40c4cb55-d1f5-407b-832f-4756b8ff77b4,6ec4d7d2-e382-47cc-94f6-c647577d4035,Maite,Calatayud,maite.calatayud@example.com,False,2025-02-09,2025-03-01,2025-03-02,14,119.96,Hotel Monte Verde,1.0,Madrid,False
3,f2ce8df5-7844-43e1-8c0f-97ce7a208f21,aed17fe8-eb18-4f69-bf08-df1afdc339c3,Tecla,Bonet,tecla.bonet@example.com,False,2025-02-08,2025-03-01,2025-03-02,38,211.53,Hotel Brisas del Mar,5.0,Madrid,False
4,57d4515a-447d-4067-afcb-9bdcf4e4e915,710a2fbb-264f-426a-aa66-f8fffd146a2d,Amílcar,Andrés,amílcar.andrés@example.com,False,2025-02-02,2025-03-01,2025-03-02,35,378.37,Hotel Camino del Sol,1.0,Madrid,False


In [77]:
# Al intentar filtrar la columna en sí, como es un booleano no nos devuelve nada, por lo que creamos una copia de la columna de tipo str
# a través de la cual podremos realizar el filtrado entre true y false.
data["col_prueba"] = data["competencia"].astype(str)
data.head()

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad,col_prueba
0,5256cc90-139b-43d2-8ec5-412495d751cf,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Feliciana,Cantón,feliciana.cantón@example.com,True,NaT,2025-03-01,2025-03-02,113,NaN,,NaN,,True
1,84fd6209-bd8d-4a92-bfe6-c68ee2c49271,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Leonardo,Hierro,leonardo.hierro@example.com,True,NaT,2025-03-01,2025-03-02,194,NaN,,NaN,,True
2,40c4cb55-d1f5-407b-832f-4756b8ff77b4,6ec4d7d2-e382-47cc-94f6-c647577d4035,Maite,Calatayud,maite.calatayud@example.com,False,2025-02-09,2025-03-01,2025-03-02,14,119.96,Hotel Monte Verde,1.0,Madrid,False
3,f2ce8df5-7844-43e1-8c0f-97ce7a208f21,aed17fe8-eb18-4f69-bf08-df1afdc339c3,Tecla,Bonet,tecla.bonet@example.com,False,2025-02-08,2025-03-01,2025-03-02,38,211.53,Hotel Brisas del Mar,5.0,Madrid,False
4,57d4515a-447d-4067-afcb-9bdcf4e4e915,710a2fbb-264f-426a-aa66-f8fffd146a2d,Amílcar,Andrés,amílcar.andrés@example.com,False,2025-02-02,2025-03-01,2025-03-02,35,378.37,Hotel Camino del Sol,1.0,Madrid,False


`Hoteles de la competencia (col True)`

Estas columnas entendemos que se consideran como competencia del grupo, por lo que nos falta información de las mismas.

Tenemos valores nulos en este caso en las columnas de inicio y fin de estancia (23 en cada una), los cuales se corresponden en ambas columnas con los mismos clientes. Asimismo hay valores nulos en la columna de precio de noche (5172), que entendemos que se corresponde con los valores nulos de los hoteles de los cuales no tenemos estrellas (nulos en columna estrellas). Existen valores nulos en la columna de fecha de reserva, de la cual no tenemos ninguna información al respecto. 

Por otro lado no tenemos los nombres de estos hoteles, sino que están puestos en el dataframe como espacios en blanco.

Para tratar estos datos realizaremos un scrapeo de los nombres de los hoteles correspondientes, sus estrellas y el precio por noche de cada uno. Una vez hayamos obtenido la información necesaria, les asignaremos un id de hotel para poder trabajar en el dataframe.


Tenemos valores duplicados en la columna de id de cliente, será necesario eliminar estos valores. 

De estos hoteles todas las estancias se han realizado entre el 01/03/2025 y el 02/03/2025. Podemos considerar rellenar el resto de fechas con los mismos valores.

In [78]:
# Filtramos por los valores que son True
hoteles_competencia = data[data["col_prueba"] == "True"]
hoteles_competencia.head()

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad,col_prueba
0,5256cc90-139b-43d2-8ec5-412495d751cf,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Feliciana,Cantón,feliciana.cantón@example.com,True,NaT,2025-03-01,2025-03-02,113,NaN,,NaN,,True
1,84fd6209-bd8d-4a92-bfe6-c68ee2c49271,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Leonardo,Hierro,leonardo.hierro@example.com,True,NaT,2025-03-01,2025-03-02,194,NaN,,NaN,,True
7,4d65096e-5f12-4ce7-9fd6-dd20e3cf5194,443a7307-2183-4051-b0bb-26f12fc1f478,Judith,Rivas,judith.rivas@example.com,True,NaT,2025-03-01,2025-03-02,131,NaN,,NaN,,True
9,458029ad-cc9b-4d85-8290-fec170bcc02d,e1a34338-a0aa-4b0e-96de-55f884eeb074,Adolfo,Cuervo,adolfo.cuervo@example.com,True,NaT,2025-03-01,2025-03-02,114,NaN,,NaN,,True
11,ac0fe86e-90a0-4fe6-a02e-6ea00e9a8621,2949fb90-39ab-4613-b4c1-e238bb56faf3,María Carmen,Navarro,maría carmen.navarro@example.com,True,NaT,2025-03-01,2025-03-02,103,NaN,,NaN,,True


In [48]:
info_df(hoteles_competencia)

,Tipo_dato,numero_registros,Numero_nulos,%_nulos
id_reserva,object,5172,0,0.00
id_cliente,object,5172,0,0.00
nombre,object,5172,0,0.00
apellido,object,5172,0,0.00
mail,object,5172,0,0.00
competencia,bool,5172,0,0.00
fecha_reserva,datetime64[ns],0,5172,100.00
inicio_estancia,datetime64[ns],5149,23,0.44
final_estancia,datetime64[ns],5149,23,0.44
id_hotel,object,5172,0,0.00


In [79]:
# Comprobamos si los valores nulos de las columnas de inicio y final de estancia se corresponden con los mismos clientes en ambas columnas.
# hoteles_competencia[hoteles_competencia["inicio_estancia"].isnull()] == hoteles_competencia[hoteles_competencia["final_estancia"].isnull()] 

In [59]:
hoteles_competencia["fecha_reserva"].unique()

<DatetimeArray>
['NaT']
Length: 1, dtype: datetime64[ns]

In [80]:
hoteles_competencia["inicio_estancia"].unique()

<DatetimeArray>
['2025-03-01 00:00:00', 'NaT']
Length: 2, dtype: datetime64[ns]

In [81]:
hoteles_competencia["final_estancia"].unique()

<DatetimeArray>
['2025-03-02 00:00:00', 'NaT']
Length: 2, dtype: datetime64[ns]

In [82]:
# Comprobamos los nombres de los hoteles de la competencia
hoteles_competencia["nombre_hotel"].unique()

array([''], dtype=object)

In [83]:
# Comprobamos los id de hotel de la competencia
hoteles_competencia["id_hotel"].unique()

array(['113', '194', '131', '114', '103', '181', '128', '186', '135',
       '117'], dtype=object)

In [84]:
hoteles_competencia.duplicated().sum()

np.int64(0)

In [85]:
# Revisamos duplicados
hoteles_competencia[hoteles_competencia["id_cliente"].duplicated()]

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad,col_prueba
1,84fd6209-bd8d-4a92-bfe6-c68ee2c49271,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Leonardo,Hierro,leonardo.hierro@example.com,True,NaT,2025-03-01,2025-03-02,194,NaN,,NaN,,True
1333,35888670-8393-4a86-b10f-42d8cc0cc558,bfbb9754-fcab-4580-bb2a-9ab295149227,Teodora,Seco,teodora.seco@example.com,True,NaT,2025-03-01,2025-03-02,128,NaN,,NaN,,True
2467,20c86be7-6537-4d1e-8553-3e95b49af0b9,37ec97b1-9ceb-48e0-a3af-7a79b9585603,Ruy,Ríos,ruy.ríos@example.com,True,NaT,2025-03-01,2025-03-02,131,NaN,,NaN,,True
3777,beda58ca-7f20-488c-b680-b6d748f666c5,2b15a636-76f1-4c9a-b689-0581c3245aa2,Ana,Valenzuela,ana.valenzuela@example.com,True,NaT,2025-03-01,2025-03-02,103,NaN,,NaN,,True
3944,3e9c3923-4b34-4566-ab96-74237b4f4eae,a44bb598-0a86-4e4a-b19e-66ddff9f3d2f,Manuela,Fonseca,manuela.fonseca@example.com,True,NaT,2025-03-01,2025-03-02,117,NaN,,NaN,,True
4826,fbba658c-f025-484c-a3a4-7a7a8af42863,21294898-0afc-489d-8f71-1ca78bedbe2f,Lina,Ariño,lina.ariño@example.com,True,NaT,2025-03-01,2025-03-02,194,NaN,,NaN,,True
8373,8b3a1b34-68dd-4429-93fa-c561ca8137c6,5a2d9076-03e5-482c-a7a4-45f5abc6cc62,Álvaro,Pascual,álvaro.pascual@example.com,True,NaT,2025-03-01,2025-03-02,103,NaN,,NaN,,True
8551,dcfaed8a-ce16-47f6-8eae-69c410a4ea8b,f3e84e3b-35a9-4207-a6a1-4cff7487f569,Consuelo,Estévez,consuelo.estévez@example.com,True,NaT,2025-03-01,2025-03-02,131,NaN,,NaN,,True
11055,9dcb96fc-ac04-4d47-b7e3-b19705c0b5bd,f8fc8162-f481-44f2-ac98-4a952cbe8ffc,Francisco Javier,Sainz,francisco javier.sainz@example.com,True,NaT,2025-03-01,2025-03-02,114,NaN,,NaN,,True
11858,9387407f-a17f-4ccb-b0b9-fc8e77b961ed,9e659fbd-bcc6-4236-9787-6bca04d567c1,Ciríaco,Viana,ciríaco.viana@example.com,True,NaT,2025-03-01,2025-03-02,181,NaN,,NaN,,True


In [95]:
# hoteles_competencia

`Hoteles del grupo (col False)`

Estas columnas entendemos que no se consideran como competencia del grupo, por lo que tenemos casi toda la información de las mismas. Tenemos los nombres de los hoteles del grupo, junto a sus id correspondientes.

Tenemos valores nulos en este caso en las columnas de inicio, fin de estancia y precio de noche (52 en cada una), los cuales se corresponden en todas las columnas con los mismos clientes.

Tenemos duplicados en las columnas de id de cliente (178) e id de reserva (98)

In [87]:
# Filtramos por las columnas que contienen False.
hoteles_grupo = data[data["col_prueba"] == "False"]
hoteles_grupo.head()

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad,col_prueba
2,40c4cb55-d1f5-407b-832f-4756b8ff77b4,6ec4d7d2-e382-47cc-94f6-c647577d4035,Maite,Calatayud,maite.calatayud@example.com,False,2025-02-09,2025-03-01,2025-03-02,14,119.96,Hotel Monte Verde,1.0,Madrid,False
3,f2ce8df5-7844-43e1-8c0f-97ce7a208f21,aed17fe8-eb18-4f69-bf08-df1afdc339c3,Tecla,Bonet,tecla.bonet@example.com,False,2025-02-08,2025-03-01,2025-03-02,38,211.53,Hotel Brisas del Mar,5.0,Madrid,False
4,57d4515a-447d-4067-afcb-9bdcf4e4e915,710a2fbb-264f-426a-aa66-f8fffd146a2d,Amílcar,Andrés,amílcar.andrés@example.com,False,2025-02-02,2025-03-01,2025-03-02,35,378.37,Hotel Camino del Sol,1.0,Madrid,False
5,ccbd9fe9-5a60-4f6f-bed3-73a7158ecba7,df42ae3d-0b9b-4c06-b22e-939323833ef8,Joan,Vazquez,joan.vazquez@example.com,False,2025-02-08,2025-03-01,2025-03-02,36,85.45,Hotel Puerta del Cielo,5.0,Madrid,False
6,99c3dc4f-663c-45f7-849a-ac9313f3746a,a16a9d83-6888-4947-a7ab-ed41203e347c,Chelo,Flor,chelo.flor@example.com,False,2025-02-09,2025-03-01,2025-03-02,43,309.80,Hotel Encanto Real,1.0,Madrid,False


In [88]:
info_df(hoteles_grupo)

,Tipo_dato,numero_registros,Numero_nulos,%_nulos
id_reserva,object,9828,0,0.00
id_cliente,object,9828,0,0.00
nombre,object,9828,0,0.00
apellido,object,9828,0,0.00
mail,object,9828,0,0.00
competencia,bool,9828,0,0.00
fecha_reserva,datetime64[ns],9828,0,0.00
inicio_estancia,datetime64[ns],9776,52,0.53
final_estancia,datetime64[ns],9776,52,0.53
id_hotel,object,9828,0,0.00


In [24]:
# Comprobamos si los valores nulos de las columnas de inicio y final de estancia se corresponden con los mismos clientes en ambas columnas.
# df_false[df_false["inicio_estancia"].isnull()] == df_false[df_false["final_estancia"].isnull()]

In [98]:
hoteles_grupo["inicio_estancia"].unique()

<DatetimeArray>
['2025-03-01 00:00:00', 'NaT']
Length: 2, dtype: datetime64[ns]

In [96]:
hoteles_grupo["final_estancia"].unique()

<DatetimeArray>
['2025-03-02 00:00:00', 'NaT']
Length: 2, dtype: datetime64[ns]

In [89]:
# Vamos a revisar los nombres de los hoteles y su id_correspondiente
hoteles_grupo[["id_hotel","nombre_hotel"]]

,id_hotel,nombre_hotel
2,14,Hotel Monte Verde
3,38,Hotel Brisas del Mar
4,35,Hotel Camino del Sol
5,36,Hotel Puerta del Cielo
6,43,Hotel Encanto Real
8,6,Palacio del Sol
10,3,Hotel Jardines del Rey
13,29,Hotel Las Estrellas
15,47,Gran Hotel Madrid
17,14,Hotel Torre Dorada


In [90]:
hoteles_grupo.duplicated().sum()

np.int64(0)

In [91]:
# Revisamos duplicados de los valores 
hoteles_grupo["id_reserva"].duplicated().sum()

np.int64(0)

In [ ]:
hoteles_grupo["id_cliente"].duplicated().sum()

np.int64(80)

In [ ]:
hoteles_grupo[hoteles_grupo["id_cliente"].duplicated()]

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad,col_prueba
557,35c7c9d8-4898-4d05-9c34-099e71348a64,e13ac13a-2b7f-4fb6-a98f-5cd417859a23,Adelardo,Bernad,adelardo.bernad@example.com,False,2025-02-01,2025-03-01,2025-03-02,33,178.04,Hotel Camino del Sol,5.0,Madrid,False
583,172dc5c0-ed32-4f1c-ac40-21f25d3e0742,cba30122-d7aa-4bca-8b57-70eb0007ff95,Martín,Villalba,martín.villalba@example.com,False,2025-02-05,2025-03-01,2025-03-02,19,82.85,Hotel Los Almendros,3.0,Madrid,False
592,d75f9705-4c2a-4f9f-a080-584a94eb72b5,614c4625-81ae-4cb8-be62-beef3ba967bb,Nayara,Vives,nayara.vives@example.com,False,2025-02-04,2025-03-01,2025-03-02,15,258.34,Hotel Camino del Sol,4.0,Madrid,False
653,47b96396-8992-40d5-a15f-0ad4c74ca2c2,f5cf6d6a-eec3-465a-a5da-f8ac77410347,Marciano,Tomas,marciano.tomas@example.com,False,2025-02-03,2025-03-01,2025-03-02,6,274.93,Hotel Los Almendros,3.0,Madrid,False
686,5b3f7171-ea11-4cd5-9e8f-037085d190c0,e444732b-9473-4bef-b72f-fe03459e4f01,Leyre,Mínguez,leyre.mínguez@example.com,False,2025-02-10,2025-03-01,2025-03-02,33,428.72,Hotel Luz de Madrid,3.0,Madrid,False
837,5c3855a1-981c-4deb-a9e1-7ec90bd337ae,0356706f-5225-4ac8-ab63-1101fa34ca9c,Clotilde,Hoyos,clotilde.hoyos@example.com,False,2025-02-06,2025-03-01,2025-03-02,48,154.84,Hotel Brisas del Mar,1.0,Madrid,False
1134,66076c28-3a21-4e4d-a115-63fd30027a4f,fc80813f-65e8-4ae7-a179-8308c1440cf8,Leonor,Hurtado,leonor.hurtado@example.com,False,2025-02-12,2025-03-01,2025-03-02,49,158.15,Hotel Costa Azul,1.0,Madrid,False
1232,2be5f54f-7531-4137-9a06-80fb65ac0dac,815583e1-7151-4095-944a-d967b0b8660d,Cintia,Barrio,cintia.barrio@example.com,False,2025-02-02,2025-03-01,2025-03-02,38,283.88,Hotel Luz de Madrid,2.0,Madrid,False
1279,e558fc57-2ccc-4610-9c81-098a7fc614a1,2dc03d93-8c75-4f34-87bc-91ba86ad31eb,Josep,Aparicio,josep.aparicio@example.com,False,2025-02-06,2025-03-01,2025-03-02,35,431.59,Hotel Brisas del Mar,5.0,Madrid,False
1331,56239530-6705-47bd-8d8c-6942cf2b06e9,fa6dca51-93f8-49ab-b968-64fad32964a7,Eutropio,Cervantes,eutropio.cervantes@example.com,False,2025-02-12,2025-03-01,2025-03-02,43,179.03,Hotel Camino del Sol,1.0,Madrid,False


## Columna id_hotel

In [29]:
# Tenemos el id del hotel como tipo int, pero nos interesa modificarlo a tipo str/object
data["id_hotel"].dtypes

dtype('int64')

In [30]:
# Realizamos la modificación del tipo de dato
data["id_hotel"] = data["id_hotel"].astype(str)

In [31]:
# Comprobamos que se ha realizado la modificación correctamente
info_df(data)

,Tipo_dato,numero_registros,Numero_nulos,%_nulos
id_reserva,object,15098,0,0.00
id_cliente,object,15098,0,0.00
nombre,object,15098,0,0.00
apellido,object,15098,0,0.00
mail,object,15098,0,0.00
competencia,bool,15098,0,0.00
fecha_reserva,datetime64[ns],9926,5172,34.26
inicio_estancia,datetime64[ns],15023,75,0.50
final_estancia,datetime64[ns],15023,75,0.50
id_hotel,object,15098,0,0.00


In [32]:
arch_limpio = data.to_pickle("../data/reservas_hoteles_limpio.pkl")
arch_limpio